In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

def main():
    print("📖 Lettura FacoDMEK.xlsx...")
    df = pd.read_excel('FacoDMEK.xlsx', sheet_name='Cleaned Data')
    
    # Salva stato originale
    original_missing = {
        'Bio-ACD': df['Bio-ACD'].isna().sum(),
        'Bio-WTW': df['Bio-WTW'].isna().sum(),
        'Bio-LT': df['Bio-LT'].isna().sum()
    }
    
    print(f"\n📊 STATO INIZIALE:")
    print(f"Totale righe: {len(df)}")
    for param, count in original_missing.items():
        print(f"{param} mancanti: {count}")
    
    # Calcola K medio
    df['K_avg'] = (df['Bio-Ks'] + df['Bio-Kf']) / 2
    
    # FASE 1: Imputa Bio-ACD (non dipende da WTW o LT)
    print("\n" + "="*60)
    print("FASE 1: IMPUTAZIONE Bio-ACD")
    print("="*60)
    
    acd_filled = 0
    for idx in df.index:
        if pd.isna(df.loc[idx, 'Bio-ACD']):
            AL = df.loc[idx, 'Bio-AL']
            K = df.loc[idx, 'K_avg']
            CCT = df.loc[idx, 'CCT']
            
            if pd.notna(AL) and pd.notna(K):
                # Formula ACD
                acd = 2.0 + 0.1 * (AL - 23.5) - 0.02 * (K - 43.5)
                
                if pd.notna(CCT):
                    acd += 0.001 * (CCT - 550)
                
                df.loc[idx, 'Bio-ACD'] = np.clip(acd, 2.0, 4.5)
                acd_filled += 1
    
    print(f"✅ Bio-ACD imputati: {acd_filled}")
    
    # FASE 2: Costruisci modello per Bio-LT basato sui dati esistenti
    print("\n" + "="*60)
    print("FASE 2: ANALISI E IMPUTAZIONE Bio-LT")
    print("="*60)
    
    # Analizza correlazioni per LT
    lt_features = ['Bio-AL', 'Bio-ACD', 'K_avg']
    complete_lt_data = df[lt_features + ['Bio-LT']].dropna()
    
    lt_model = None
    lt_filled = 0
    
    if len(complete_lt_data) > 15:
        X_lt = complete_lt_data[lt_features]
        y_lt = complete_lt_data['Bio-LT']
        
        lt_model = LinearRegression()
        lt_model.fit(X_lt, y_lt)
        r2_lt = lt_model.score(X_lt, y_lt)
        
        print(f"Modello Bio-LT creato (R² = {r2_lt:.3f}):")
        print(f"Bio-LT = {lt_model.intercept_:.3f}", end="")
        for feat, coef in zip(lt_features, lt_model.coef_):
            print(f" + {coef:.4f}×{feat}" if coef > 0 else f" - {abs(coef):.4f}×{feat}", end="")
        print()
        
        # Applica imputazione LT
        for idx in df.index:
            if pd.isna(df.loc[idx, 'Bio-LT']):
                if all(pd.notna(df.loc[idx, feat]) for feat in lt_features):
                    X_pred = df.loc[idx, lt_features].values.reshape(1, -1)
                    pred_lt = lt_model.predict(X_pred)[0]
                    df.loc[idx, 'Bio-LT'] = np.clip(pred_lt, 3.0, 6.0)
                    lt_filled += 1
    else:
        print("⚠️ Dati insufficienti per creare modello Bio-LT")
    
    print(f"✅ Bio-LT imputati: {lt_filled}")
    
    # FASE 3: Imputa Bio-WTW (può usare ACD e LT imputati)
    print("\n" + "="*60)
    print("FASE 3: IMPUTAZIONE Bio-WTW")
    print("="*60)
    
    # Prima prova a creare modello basato sui dati
    wtw_features = ['K_avg', 'Bio-AL', 'Bio-ACD']
    complete_wtw_data = df[wtw_features + ['Bio-WTW']].dropna()
    
    wtw_model = None
    wtw_filled = 0
    
    if len(complete_wtw_data) > 15:
        X_wtw = complete_wtw_data[wtw_features]
        y_wtw = complete_wtw_data['Bio-WTW']
        
        wtw_model = LinearRegression()
        wtw_model.fit(X_wtw, y_wtw)
        r2_wtw = wtw_model.score(X_wtw, y_wtw)
        
        print(f"Modello Bio-WTW creato (R² = {r2_wtw:.3f}):")
        print(f"Bio-WTW = {wtw_model.intercept_:.3f}", end="")
        for feat, coef in zip(wtw_features, wtw_model.coef_):
            print(f" + {coef:.4f}×{feat}" if coef > 0 else f" - {abs(coef):.4f}×{feat}", end="")
        print()
    
    # Applica imputazione WTW
    for idx in df.index:
        if pd.isna(df.loc[idx, 'Bio-WTW']):
            # Prima prova con modello se disponibile
            if wtw_model and all(pd.notna(df.loc[idx, feat]) for feat in wtw_features):
                X_pred = df.loc[idx, wtw_features].values.reshape(1, -1)
                pred_wtw = wtw_model.predict(X_pred)[0]
                df.loc[idx, 'Bio-WTW'] = np.clip(pred_wtw, 10.5, 13.0)
                wtw_filled += 1
            # Altrimenti usa formula standard
            elif pd.notna(df.loc[idx, 'K_avg']):
                K_avg = df.loc[idx, 'K_avg']
                AL = df.loc[idx, 'Bio-AL']
                ACD = df.loc[idx, 'Bio-ACD']
                
                wtw = 11.8 - 0.05 * (K_avg - 43.5)
                
                if pd.notna(AL):
                    wtw += 0.03 * (AL - 23.5)
                
                if pd.notna(ACD):
                    wtw += 0.2 * (ACD - 3.0)
                
                df.loc[idx, 'Bio-WTW'] = np.clip(wtw, 10.5, 13.0)
                wtw_filled += 1
    
    print(f"✅ Bio-WTW imputati: {wtw_filled}")
    
    # Rimuovi colonna temporanea
    df = df.drop(columns=['K_avg'])
    
    # RISULTATI FINALI
    print("\n" + "="*60)
    print("📊 RISULTATI FINALI:")
    print("="*60)
    
    final_missing = {
        'Bio-ACD': df['Bio-ACD'].isna().sum(),
        'Bio-WTW': df['Bio-WTW'].isna().sum(),
        'Bio-LT': df['Bio-LT'].isna().sum()
    }
    
    summary_data = []
    for param in ['Bio-ACD', 'Bio-WTW', 'Bio-LT']:
        summary_data.append({
            'Parametro': param,
            'Mancanti_Iniziali': original_missing[param],
            'Imputati': original_missing[param] - final_missing[param],
            'Ancora_Mancanti': final_missing[param],
            'Percentuale_Completezza': f"{(1 - final_missing[param]/len(df))*100:.1f}%"
        })
    
    summary_df = pd.DataFrame(summary_data)
    print(summary_df.to_string(index=False))
    
    # Statistiche dei parametri
    print(f"\n📈 STATISTICHE FINALI:")
    for param in ['Bio-ACD', 'Bio-WTW', 'Bio-LT']:
        if df[param].notna().sum() > 0:
            print(f"{param}: Media={df[param].mean():.3f}, Min={df[param].min():.2f}, Max={df[param].max():.2f}")
    
    # Salva file principale
    output_file = 'FacoDMEK_Completo_Tripla_Imputazione.xlsx'
    df.to_excel(output_file, sheet_name='Cleaned Data', index=False)
    print(f"\n✅ FILE PRINCIPALE SALVATO: {output_file}")
    
    # Crea report dettagliato
    print("\n📄 Creazione report dettagliato...")
    
    # Rileggi originale per confronto
    df_original = pd.read_excel('FacoDMEK.xlsx', sheet_name='Cleaned Data')
    
    # Identifica valori imputati
    imputed_masks = {
        'ACD': df_original['Bio-ACD'].isna() & df['Bio-ACD'].notna(),
        'WTW': df_original['Bio-WTW'].isna() & df['Bio-WTW'].notna(),
        'LT': df_original['Bio-LT'].isna() & df['Bio-LT'].notna()
    }
    
    # Crea report con flag
    df_report = df.copy()
    df_report['ACD_Imputato'] = 'No'
    df_report['WTW_Imputato'] = 'No'
    df_report['LT_Imputato'] = 'No'
    
    df_report.loc[imputed_masks['ACD'], 'ACD_Imputato'] = 'Sì'
    df_report.loc[imputed_masks['WTW'], 'WTW_Imputato'] = 'Sì'
    df_report.loc[imputed_masks['LT'], 'LT_Imputato'] = 'Sì'
    
    # Salva report Excel multi-foglio
    with pd.ExcelWriter('Report_Tripla_Imputazione.xlsx') as writer:
        # Foglio 1: Riepilogo
        summary_df.to_excel(writer, sheet_name='Riepilogo', index=False)
        
        # Foglio 2: Formule utilizzate
        formulas = pd.DataFrame([
            {
                'Parametro': 'Bio-ACD',
                'Formula': 'ACD = 2.0 + 0.1×(AL-23.5) - 0.02×(K-43.5) + 0.001×(CCT-550)',
                'Note': 'Formula standard, range 2.0-4.5 mm'
            },
            {
                'Parametro': 'Bio-LT',
                'Formula': f'Basata su regressione dai dati' if lt_model else 'Non disponibile',
                'Note': f'R² = {r2_lt:.3f}' if lt_model else 'Dati insufficienti'
            },
            {
                'Parametro': 'Bio-WTW',
                'Formula': f'Basata su regressione dai dati' if wtw_model else 'Formula standard',
                'Note': f'R² = {r2_wtw:.3f}' if wtw_model else 'WTW = 11.8 - 0.05×(K-43.5) + correzioni'
            }
        ])
        formulas.to_excel(writer, sheet_name='Formule', index=False)
        
        # Foglio 3: Solo righe con imputazioni
        any_imputed = imputed_masks['ACD'] | imputed_masks['WTW'] | imputed_masks['LT']
        if any_imputed.sum() > 0:
            imputed_rows = df_report[any_imputed][
                ['ID', 'Patient', 'Eye', 'Bio-AL', 'Bio-ACD', 'ACD_Imputato',
                 'Bio-WTW', 'WTW_Imputato', 'Bio-LT', 'LT_Imputato']
            ]
            imputed_rows.to_excel(writer, sheet_name='Righe Imputate', index=False)
        
        # Foglio 4: Statistiche per parametro
        stats_data = []
        for param, mask in imputed_masks.items():
            if mask.sum() > 0:
                values = df.loc[mask, f'Bio-{param}']
                stats_data.append({
                    'Parametro': f'Bio-{param}',
                    'N_Imputati': mask.sum(),
                    'Media': values.mean(),
                    'Mediana': values.median(),
                    'Dev_Std': values.std(),
                    'Min': values.min(),
                    'Max': values.max()
                })
        
        if stats_data:
            pd.DataFrame(stats_data).to_excel(writer, sheet_name='Statistiche Imputati', index=False)
    
    print("📊 Report dettagliato salvato in: Report_Tripla_Imputazione.xlsx")
    
    print("\n" + "="*60)
    print("✨ PROCESSO COMPLETATO!")
    print("="*60)
    print("\nTutti i parametri biometrici sono stati imputati dove possibile.")
    print("I modelli di regressione sono stati costruiti basandosi esclusivamente")
    print("sulle correlazioni presenti nei tuoi dati FacoDMEK.")

if __name__ == "__main__":
    main()

📖 Lettura FacoDMEK.xlsx...

📊 STATO INIZIALE:
Totale righe: 97
Bio-ACD mancanti: 0
Bio-WTW mancanti: 0
Bio-LT mancanti: 7

FASE 1: IMPUTAZIONE Bio-ACD
✅ Bio-ACD imputati: 0

FASE 2: ANALISI E IMPUTAZIONE Bio-LT
Modello Bio-LT creato (R² = 0.181):
Bio-LT = 6.082 + 0.0200×Bio-AL - 0.3872×Bio-ACD - 0.0184×K_avg
✅ Bio-LT imputati: 7

FASE 3: IMPUTAZIONE Bio-WTW
Modello Bio-WTW creato (R² = 0.143):
Bio-WTW = 14.250 - 0.0755×K_avg + 0.0122×Bio-AL + 0.2402×Bio-ACD
✅ Bio-WTW imputati: 0

📊 RISULTATI FINALI:
Parametro  Mancanti_Iniziali  Imputati  Ancora_Mancanti Percentuale_Completezza
  Bio-ACD                  0         0                0                  100.0%
  Bio-WTW                  0         0                0                  100.0%
   Bio-LT                  7         7                0                  100.0%

📈 STATISTICHE FINALI:
Bio-ACD: Media=3.040, Min=1.88, Max=3.95
Bio-WTW: Media=11.991, Min=10.64, Max=13.67
Bio-LT: Media=4.584, Min=3.44, Max=5.56

✅ FILE PRINCIPALE SALVATO: